# Classifying X-Rays using CNN
The goal is to use a model to classify X-Rays from the NIH lung x-ray dataset and Kaggle Covid-19 x-ray dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
all_xray_df = pd.read_csv('Data_Entry_2017_v2020.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('data','*'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)

# Preprocessing Labels
Here we take the labels and make them into a more clear format. The primary step is to see the distribution of findings and then to convert them to simple binary labels. We have two different runs: 1.) exclude no finding 2.) include no finding. We use this process to filter out unwanted labels and get quantitative data on our dataset.

In [ ]:
label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
# uncomment the line below and comment out the line above if you want to include no Finding
#
# all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', 'No_Finding'))

all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: '' if (len(x.split('|')) > 1) else x )
from itertools import chain
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        #setting binary labels for each image id 'one' if the current 'c_label', zero otherwise.
        all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
all_xray_df = all_xray_df[all_xray_df['Finding Labels']!='']

all_xray_df.sample(10)


### Clean categories
Using this method we remove labels with a minimum amount of cases. This comes in handy when removing labels that have more than one classification, or not enough images to train the model with.

In [ ]:
# keep at least 100 cases
MIN_CASES = 100
all_labels = [c_label for c_label in all_labels if all_xray_df[c_label].sum()>MIN_CASES]
print('Clean Labels ({})'.format(len(all_labels)), 
      [(c_label,int(all_xray_df[c_label].sum())) for c_label in all_labels])

In [ ]:
# since the dataset is very unbiased, we can resample it to be a more reasonable collection
# weight is 0.1 + number of findings
sample_weights = all_xray_df['Finding Labels'].map(lambda x: len(x.split('|')) if len(x)>0 else 0).values + 4e-2
sample_weights /= sample_weights.sum()
all_xray_df = all_xray_df.sample(31178, weights=sample_weights)

label_counts = all_xray_df['Finding Labels'].value_counts()[:]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
label_counts = 100*np.mean(all_xray_df[all_labels].values,0)
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
ax1.set_xticklabels(all_labels, rotation = 90)
ax1.set_title('Adjusted Frequency of Diseases in Patient Group')
_ = ax1.set_ylabel('Frequency (%)')

# Preparing Training and Testing Data
Here we split the data into training and validation sets and create a single vector (disease_vec) with the 0/1 outputs for the disease status (what the model will try and predict). Given the 0 and 1 outputs we will use a binary classification model to train our CNN. We felt after testing with sparse categorical accuracy, we found that our results were not great. Switching to binary our results improved drastically.

In [ ]:
all_xray_df['disease_vec'] = all_xray_df.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])


In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(all_xray_df, 
                                   test_size = 0.25,
                                   stratify = all_xray_df['Finding Labels'].map(lambda x: x[:4]))
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

# Build Data
We build the data generator to randomly transform our images and categorize them into training, validtaion, and testing.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (256, 256)
core_idg = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.05, 
                              width_shift_range=0.1, 
                              rotation_range=10, 
                              shear_range = 0.1,
                              fill_mode = 'nearest',
                              zoom_range=0.10)

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [ ]:
train_gen = flow_from_dataframe(core_idg, train_df, 
                             path_col = 'path',
                            y_col = 'disease_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'grayscale',
                            batch_size = 50)

valid_gen = flow_from_dataframe(core_idg, valid_df, 
                             path_col = 'path',
                            y_col = 'disease_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'grayscale',
                            batch_size = 256) # we can use much larger batches for evaluation
# used a fixed dataset for evaluating the algorithm
test_X, test_Y = next(flow_from_dataframe(core_idg, 
                               valid_df, 
                             path_col = 'path',
                            y_col = 'disease_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'grayscale',
                            batch_size = 1024)) # one big batch

In [ ]:
#  This is going to display how some of our images changed using our data generator.
t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = -1.5, vmax = 1.5)
    c_ax.set_title(', '.join([n_class for n_class, n_score in zip(all_labels, c_y) 
                             if n_score>0.5]))
    c_ax.axis('off')

# Creating model
Here we build two models. One is our design, having read numerous articles online about x-ray classification. Another is a well established keras Architecture called MobileNet. We would have liked to run this on densenet121, or ResnNet, however given computing restrictions, we were not able to do so.

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
# UnComment this if you want to run our model 
#
# multi_disease_model = Sequential([
#             #Block 1
#             Conv2D(32, 3, strides=1, padding="same", activation="relu", name="block1_conv1", input_shape=(256,256, 1)),
#             Conv2D(32, 3, strides=1, padding="same", activation="relu", name="block1_conv2"),
#             MaxPool2D(4, name="block1_pool"),
#             #Block 2
#             Conv2D(64, 3, strides=1, padding="same", activation="relu", name="block2_conv1"),
#             Conv2D(64, 3, strides=1, padding="same", activation="relu", name="block2_conv2"),
#             MaxPool2D(2, name="block2_pool"),
#             # Block 3
#             Conv2D(128, 5, strides=1, padding="same", activation="relu", name="block3_conv1"),
#             Conv2D(128, 5, strides=1, padding="same", activation="relu", name="block3_conv2"),
#             Conv2D(128, 5, strides=1, padding="same", activation="relu", name="block3_conv3"),
#             MaxPool2D(2, name="block3_pool"),
#              #Block 4
#             Conv2D(256, 5, strides=1, padding="same", activation="relu", name="block4_conv1"),
#             Conv2D(256, 5, strides=1, padding="same", activation="relu", name="block4_conv2"),
#             Conv2D(256, 5, strides=1, padding="same", activation="relu", name="block4_conv3")

# ])
multi_disease_model = Sequential()
# Comment the line below if you want to run our model
multi_disease_model.add(MobileNet(weights = None, input_shape=(256,256, 1), classes=1000))
multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Flatten())
multi_disease_model.add(Dense(512, activation='relu'))
multi_disease_model.add(Dropout(0.25))
multi_disease_model.add(Dense(len(all_labels), activation = 'sigmoid'))

multi_disease_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae'])
multi_disease_model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

weight_path="weights/xray_weights_best.hdf5"


checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=3)
# Several articles we read that classify x-rays use this callback and it helped improve their accuracy and so we decided to implement this.
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')

callbacks_list = [checkpoint, early, reduceLR]

# First Epoch
Here we decided to test our first epoch and see that our model has progressed and learned and gives some good initial feedback to determine if the model is sufficient.

In [ ]:
multi_disease_model.fit_generator(train_gen, 
                                  steps_per_epoch=200,
                                  validation_data = (test_X, test_Y), 
                                  epochs = 1, 
                                  callbacks = callbacks_list)

# Testing First Epoch
Here you could see the predicted frequency of each label. 

In [ ]:
for c_label, s_count in zip(all_labels, 100*np.mean(test_Y,0)):
    print('%s: %2.2f%%' % (c_label, s_count))

In [ ]:
pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)

# ROC Curves
We can show the ROC curves for each label and believe it is a good metric to measure reliability of a binary classifier, while accuracy alone might not be the best metric for testing how well our model predicts. 

In [ ]:
from sklearn.metrics import roc_curve, auc
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (idx, c_label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(test_Y[:,idx].astype(int), pred_Y[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('barely_trained_net.png')

# Continued Training
we want to test how our model does after extensive training and compare our results with our first epoc

In [ ]:
multi_disease_model.fit_generator(train_gen, 
                                  steps_per_epoch = 160,
                                  validation_data =  (test_X, test_Y), 
                                  epochs = 15, 
                                  callbacks = callbacks_list)

In [ ]:
# load the best weights
multi_disease_model.load_weights(weight_path)

In [ ]:
pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)

In [ ]:
# look at how often the algorithm predicts certain diagnoses 
for c_label, p_count, t_count in zip(all_labels, 
                                     100*np.mean(pred_Y,0), 
                                     100*np.mean(test_Y,0)):
    print('%s: Dx: %2.2f%%, PDx: %2.2f%%' % (c_label, t_count, p_count))

In [ ]:
from sklearn.metrics import roc_curve, auc
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (idx, c_label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(test_Y[:,idx].astype(int), pred_Y[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('trained_net.png')

# Show a few images and associated predictions

In [ ]:
sickest_idx = np.argsort(np.sum(test_Y, 1)<1)
fig, m_axs = plt.subplots(4, 2, figsize = (16, 32))
for (idx, c_ax) in zip(sickest_idx, m_axs.flatten()):
    c_ax.imshow(test_X[idx, :,:,0], cmap = 'bone')
    stat_str = [n_class[:6] for n_class, n_score in zip(all_labels, 
                                                                  test_Y[idx]) 
                             if n_score>0.5]
    pred_str = ['%s:%2.0f%%' % (n_class[:4], p_score*100)  for n_class, n_score, p_score in zip(all_labels, 
                                                                  test_Y[idx], pred_Y[idx]) 
                             if (n_score>0.5) or (p_score>0.5)]
    c_ax.set_title('Dx: '+', '.join(stat_str)+'\nPDx: '+', '.join(pred_str))
    c_ax.axis('off')
fig.savefig('trained_img_predictions.png')